In [9]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import datetime as dt
import re
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

In [2]:
data = pd.read_csv("bmore.csv")

In [3]:
data.head()

,Unnamed: 0,date,author,twitter_name,text,number_of_likes,number_of_retweets
0,0.0,2022-11-18 02:52:03+00:00,Brandon M. Scott,MayorBMScott,Speaker Pelosi may represent the people of San...,36,4
1,1.0,2022-11-18 02:52:02+00:00,Brandon M. Scott,MayorBMScott,So many of my peers can thank her for knocking...,26,2
2,2.0,2022-11-18 02:52:02+00:00,Brandon M. Scott,MayorBMScott,"In addition to being the first female speaker,...",9,0
3,3.0,2022-11-18 02:52:01+00:00,Brandon M. Scott,MayorBMScott,It brings us all great pride that a daughter o...,11,2
4,4.0,2022-11-18 02:52:00+00:00,Brandon M. Scott,MayorBMScott,"On behalf of the City of Baltimore, I want to ...",221,12


In [6]:
sentence = "I love hamburgers."

In [7]:
analysisPol = TextBlob(sentence).polarity
analysisSub = TextBlob(sentence).subjectivity

print(analysisPol)
print(analysisSub)

0.5
0.6


In [10]:
#function to clean @usernames, hashtags, numbers, etc., from tweets
def clean_text(text):  
    pat1 = r'@[^ ]+'                   
    pat2 = r'https?://[A-Za-z0-9./]+'  
    pat3 = r'\'s'                      
    pat4 = r'\#\w+'                     
    pat5 = r'&amp '                     
    pat6 = r'[^A-Za-z\s]'               
    combined_pat = r'|'.join((pat1, pat2,pat3,pat4,pat5, pat6))
    text = re.sub(combined_pat,"",text).lower()
    return text.strip()

In [11]:
data["cleaned_tweet"] = data["text"].apply(clean_text)

In [12]:
data.head()

,Unnamed: 0,date,author,twitter_name,text,number_of_likes,number_of_retweets,cleaned_tweet
0,0.0,2022-11-18 02:52:03+00:00,Brandon M. Scott,MayorBMScott,Speaker Pelosi may represent the people of San...,36,4,speaker pelosi may represent the people of san...
1,1.0,2022-11-18 02:52:02+00:00,Brandon M. Scott,MayorBMScott,So many of my peers can thank her for knocking...,26,2,so many of my peers can thank her for knocking...
2,2.0,2022-11-18 02:52:02+00:00,Brandon M. Scott,MayorBMScott,"In addition to being the first female speaker,...",9,0,in addition to being the first female speaker ...
3,3.0,2022-11-18 02:52:01+00:00,Brandon M. Scott,MayorBMScott,It brings us all great pride that a daughter o...,11,2,it brings us all great pride that a daughter o...
4,4.0,2022-11-18 02:52:00+00:00,Brandon M. Scott,MayorBMScott,"On behalf of the City of Baltimore, I want to ...",221,12,on behalf of the city of baltimore i want to c...


In [13]:
#runs sentiment analysis with TextBlob
#creates new "Sentiment" column where tweet is labeled positve, negative, or neutral

print("Running sentiment process")
for row in data.itertuples():
    tweet = data.at[row[0], 'cleaned_tweet']
#run sentiment using TextBlob
    analysis = TextBlob(tweet)
#set value to dataframe
    data.at[row[0], 'polarity'] = analysis.sentiment[0]
    data.at[row[0], 'subjectivity'] = analysis.sentiment[1]
#Create Positive / negative column depending on polarity
    if analysis.sentiment[0]>0:
        data.at[row[0], 'Sentiment'] = "Positive"
    elif analysis.sentiment[0]<0:
        data.at[row[0], 'Sentiment'] = "Negative"
    else:
        data.at[row[0], 'Sentiment'] = "Neutral"

Running sentiment process


In [14]:
data.head()

,Unnamed: 0,date,author,twitter_name,text,number_of_likes,number_of_retweets,cleaned_tweet,polarity,subjectivity,Sentiment
0,0.0,2022-11-18 02:52:03+00:00,Brandon M. Scott,MayorBMScott,Speaker Pelosi may represent the people of San...,36,4,speaker pelosi may represent the people of san...,0.000000,0.000000,Neutral
1,1.0,2022-11-18 02:52:02+00:00,Brandon M. Scott,MayorBMScott,So many of my peers can thank her for knocking...,26,2,so many of my peers can thank her for knocking...,0.211111,0.572222,Positive
2,2.0,2022-11-18 02:52:02+00:00,Brandon M. Scott,MayorBMScott,"In addition to being the first female speaker,...",9,0,in addition to being the first female speaker ...,0.210000,0.400000,Positive
3,3.0,2022-11-18 02:52:01+00:00,Brandon M. Scott,MayorBMScott,It brings us all great pride that a daughter o...,11,2,it brings us all great pride that a daughter o...,0.266667,0.416667,Positive
4,4.0,2022-11-18 02:52:00+00:00,Brandon M. Scott,MayorBMScott,"On behalf of the City of Baltimore, I want to ...",221,12,on behalf of the city of baltimore i want to c...,0.000000,0.000000,Neutral


In [15]:
data.to_csv('textblob_bmore.csv')

In [17]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
sentence = "The food was great!"
vs = analyzer.polarity_scores(sentence)
print("{:-<65} {}".format(sentence, str(vs)))

The food was great!---------------------------------------------- {'neg': 0.0, 'neu': 0.406, 'pos': 0.594, 'compound': 0.6588}


In [18]:
data2 = pd.read_csv("bmore.csv")

In [20]:
data2["cleaned_tweet"] = data2["text"].apply(clean_text)

In [23]:
scores = []

# Declare variables for scores*
compound_list = []
positive_list = []
negative_list = []
neutral_list = []

for i in range(data2['cleaned_tweet'].shape[0]):

    compound = analyzer.polarity_scores(data2['cleaned_tweet'][i])["compound"]
    pos = analyzer.polarity_scores(data2['cleaned_tweet'][i])["pos"]
    neu = analyzer.polarity_scores(data2['cleaned_tweet'][i])["neu"]
    neg = analyzer.polarity_scores(data2['cleaned_tweet'][i])["neg"]

    scores.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg,
                       "Neutral": neu
                  })

In [24]:
sentiments_score = pd.DataFrame.from_dict(scores)
data2 = data2.join(sentiments_score)
data2.head()

,Unnamed: 0,date,author,twitter_name,text,number_of_likes,number_of_retweets,cleaned_tweet,Compound,Positive,Negative,Neutral
0,0.0,2022-11-18 02:52:03+00:00,Brandon M. Scott,MayorBMScott,Speaker Pelosi may represent the people of San...,36,4,speaker pelosi may represent the people of san...,0.0000,0.000,0.0,1.000
1,1.0,2022-11-18 02:52:02+00:00,Brandon M. Scott,MayorBMScott,So many of my peers can thank her for knocking...,26,2,so many of my peers can thank her for knocking...,0.5106,0.185,0.0,0.815
2,2.0,2022-11-18 02:52:02+00:00,Brandon M. Scott,MayorBMScott,"In addition to being the first female speaker,...",9,0,in addition to being the first female speaker ...,0.4754,0.140,0.0,0.860
3,3.0,2022-11-18 02:52:01+00:00,Brandon M. Scott,MayorBMScott,It brings us all great pride that a daughter o...,11,2,it brings us all great pride that a daughter o...,0.7579,0.245,0.0,0.755
4,4.0,2022-11-18 02:52:00+00:00,Brandon M. Scott,MayorBMScott,"On behalf of the City of Baltimore, I want to ...",221,12,on behalf of the city of baltimore i want to c...,0.5423,0.220,0.0,0.780


In [25]:
data2.to_csv('vader_bmore.csv')